In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__


'2.15.0'

In [2]:

NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [6]:


print(f'train_data shape: {train_data.shape}')
print(f'test_data shape: {test_data.shape}')



train_data shape: (25000,)
test_data shape: (25000,)


In [7]:

print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:

word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))

1641221/1641221 [==============================] - 0s 0us/step
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole fi

In [6]:

train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [3]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(25000, 10000)

In [9]:

baseline_model = Sequential()
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                160016    
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160305 (626.19 KB)
Trainable params: 160305 (626.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))


Epoch 1/20
49/49 [==============================] - 6s 109ms/step - loss: 0.4540 - accuracy: 0.8094 - binary_crossentropy: 0.4540 - val_loss: 0.3095 - val_accuracy: 0.8822 - val_binary_crossentropy: 0.3095
Epoch 2/20
49/49 [==============================] - 3s 55ms/step - loss: 0.2264 - accuracy: 0.9165 - binary_crossentropy: 0.2264 - val_loss: 0.2923 - val_accuracy: 0.8811 - val_binary_crossentropy: 0.2923
Epoch 3/20
49/49 [==============================] - 2s 40ms/step - loss: 0.1654 - accuracy: 0.9420 - binary_crossentropy: 0.1654 - val_loss: 0.3098 - val_accuracy: 0.8783 - val_binary_crossentropy: 0.3098
Epoch 4/20
49/49 [==============================] - 3s 52ms/step - loss: 0.1321 - accuracy: 0.9550 - binary_crossentropy: 0.1321 - val_loss: 0.3290 - val_accuracy: 0.8758 - val_binary_crossentropy: 0.3290
Epoch 5/20
49/49 [==============================] - 2s 40ms/step - loss: 0.1070 - accuracy: 0.9657 - binary_crossentropy: 0.1070 - val_loss: 0.3711 - val_accuracy: 0.8680 - val_bi

In [11]:
smaller_model = Sequential()
smaller_model.add(Dense(units=4, input_shape=(NUM_WORDS,), activation='relu'))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy', 'binary_crossentropy'])

smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))


Epoch 1/20
49/49 [==============================] - 6s 99ms/step - loss: 0.6197 - accuracy: 0.6529 - binary_crossentropy: 0.6197 - val_loss: 0.5625 - val_accuracy: 0.7406 - val_binary_crossentropy: 0.5625
Epoch 2/20
49/49 [==============================] - 3s 52ms/step - loss: 0.5156 - accuracy: 0.8114 - binary_crossentropy: 0.5156 - val_loss: 0.5081 - val_accuracy: 0.8060 - val_binary_crossentropy: 0.5081
Epoch 3/20
49/49 [==============================] - 2s 42ms/step - loss: 0.4621 - accuracy: 0.8725 - binary_crossentropy: 0.4621 - val_loss: 0.4709 - val_accuracy: 0.8584 - val_binary_crossentropy: 0.4709
Epoch 4/20
49/49 [==============================] - 3s 57ms/step - loss: 0.4199 - accuracy: 0.9064 - binary_crossentropy: 0.4199 - val_loss: 0.4520 - val_accuracy: 0.8633 - val_binary_crossentropy: 0.4520
Epoch 5/20
49/49 [==============================] - 2s 40ms/step - loss: 0.3878 - accuracy: 0.9274 - binary_crossentropy: 0.3878 - val_loss: 0.4423 - val_accuracy: 0.8660 - val_bin

In [12]:
bigger_model = Sequential()
bigger_model.add(Dense(units=512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy', 'binary_crossentropy'])
bigger_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               5120512   
                                                                 
 dense_7 (Dense)             (None, 512)               262656    
                                                                 
 dense_8 (Dense)             (None, 1)                 513       
                                                                 
Total params: 5383681 (20.54 MB)
Trainable params: 5383681 (20.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 23s 460ms/step - loss: 0.3404 - accuracy: 0.8584 - binary_crossentropy: 0.3404 - val_loss: 0.2896 - val_accuracy: 0.8827 - val_binary_crossentropy: 0.2896
Epoch 2/20
49/49 [==============================] - 19s 388ms/step - loss: 0.1471 - accuracy: 0.9456 - binary_crossentropy: 0.1471 - val_loss: 0.3271 - val_accuracy: 0.8749 - val_binary_crossentropy: 0.3271
Epoch 3/20
49/49 [==============================] - 24s 492ms/step - loss: 0.0500 - accuracy: 0.9848 - binary_crossentropy: 0.0500 - val_loss: 0.4195 - val_accuracy: 0.8698 - val_binary_crossentropy: 0.4195
Epoch 4/20
49/49 [==============================] - 24s 498ms/step - loss: 0.0074 - accuracy: 0.9988 - binary_crossentropy: 0.0074 - val_loss: 0.5803 - val_accuracy: 0.8692 - val_binary_crossentropy: 0.5803
Epoch 5/20
49/49 [==============================] - 19s 400ms/step - loss: 0.0012 - accuracy: 1.0000 - binary_crossentropy: 0.0012 - val_loss: 0.6700 - val_accuracy: 0.8707

In [13]:

hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.454047,0.80944,0.454047,0.309460,0.88224,0.309460,0
1,0.226410,0.91648,0.226410,0.292299,0.88112,0.292299,1
2,0.165371,0.94204,0.165371,0.309776,0.87828,0.309776,2
3,0.132098,0.95500,0.132098,0.329046,0.87580,0.329046,3
4,0.107016,0.96568,0.107016,0.371121,0.86800,0.371121,4


In [7]:

import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline'], [smaller_history, baseline_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

NameError: name 'smaller_history' is not defined

In [4]:

from tensorflow.keras.regularizers import l2

import plotly.graph_objects as go
l2_model = Sequential()
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                160016    
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160305 (626.19 KB)
Trainable params: 160305 (626.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))


Epoch 1/20
49/49 [==============================] - 6s 109ms/step - loss: 0.6860 - accuracy: 0.7878 - binary_crossentropy: 0.5196 - val_loss: 0.5278 - val_accuracy: 0.8718 - val_binary_crossentropy: 0.3751
Epoch 2/20
49/49 [==============================] - 3s 58ms/step - loss: 0.4364 - accuracy: 0.9022 - binary_crossentropy: 0.2888 - val_loss: 0.4373 - val_accuracy: 0.8873 - val_binary_crossentropy: 0.2959
Epoch 3/20
49/49 [==============================] - 2s 41ms/step - loss: 0.3592 - accuracy: 0.9231 - binary_crossentropy: 0.2233 - val_loss: 0.4148 - val_accuracy: 0.8853 - val_binary_crossentropy: 0.2854
Epoch 4/20
49/49 [==============================] - 3s 54ms/step - loss: 0.3197 - accuracy: 0.9337 - binary_crossentropy: 0.1950 - val_loss: 0.4033 - val_accuracy: 0.8861 - val_binary_crossentropy: 0.2839
Epoch 5/20
49/49 [==============================] - 3s 54ms/step - loss: 0.2934 - accuracy: 0.9420 - binary_crossentropy: 0.1776 - val_loss: 0.3993 - val_accuracy: 0.8836 - val_bi

In [8]:
import plotly.graph_objects as go
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

NameError: name 'baseline_history' is not defined

In [9]:

from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                160016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160305 (626.19 KB)
Trainable params: 160305 (626.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))


NameError: name 'dropout_model' is not defined

In [6]:

fig = go.Figure()
for name, history in zip(['l2', 'dropout'], [l2_model_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

NameError: name 'dropout_history' is not defined

In [9]:
from tensorflow.keras.layers import Dropout
dropoutl2_model = Sequential()
dropoutl2_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,), kernel_regularizer=l2(0.001)))
dropoutl2_model.add(Dropout(0.5))
dropoutl2_model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01)))
dropoutl2_model.add(Dropout(0.5))
dropoutl2_model.add(Dense(1, activation='sigmoid'))

dropoutl2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropoutl2_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                160016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160305 (626.19 KB)
Trainable params: 160305 (626.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
l2dropout_history = dropoutl2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 [==============================] - 7s 120ms/step - loss: 0.7810 - accuracy: 0.6633 - binary_crossentropy: 0.6097 - val_loss: 0.6023 - val_accuracy: 0.8579 - val_binary_crossentropy: 0.4494
Epoch 2/20
49/49 [==============================] - 2s 40ms/step - loss: 0.5865 - accuracy: 0.8159 - binary_crossentropy: 0.4420 - val_loss: 0.4741 - val_accuracy: 0.8818 - val_binary_crossentropy: 0.3373
Epoch 3/20
49/49 [==============================] - 2s 41ms/step - loss: 0.4894 - accuracy: 0.8654 - binary_crossentropy: 0.3589 - val_loss: 0.4222 - val_accuracy: 0.8873 - val_binary_crossentropy: 0.2979
Epoch 4/20
49/49 [==============================] - 3s 55ms/step - loss: 0.4315 - accuracy: 0.8853 - binary_crossentropy: 0.3117 - val_loss: 0.3988 - val_accuracy: 0.8862 - val_binary_crossentropy: 0.2839
Epoch 5/20
49/49 [==============================] - 3s 66ms/step - loss: 0.3903 - accuracy: 0.9044 - binary_crossentropy: 0.2795 - val_loss: 0.3854 - val_accuracy: 0.8870 - val_bi

In [11]:
fig = go.Figure()
for name, history in zip(['l2', 'l2dropout'], [l2_model_history, l2dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()